In [1]:
import torch, math
import pandas as pd
from IPython.display import display

torch.set_default_dtype(torch.float64)

# ========= Utilitários =========
def grad_u(u_func, x):
    """Gradiente do campo de deslocamentos J_ij = ∂u_i/∂x_j em x."""
    x_ = x.clone().detach().requires_grad_(True)
    u = u_func(x_)
    if u.ndim != 1:
        raise ValueError('u(x) deve retornar vetor 1D (dimensão do espaço).')
    rows = []
    for i in range(u.numel()):
        gi, = torch.autograd.grad(u[i], x_, retain_graph=True, create_graph=True)
        rows.append(gi)
    J = torch.stack(rows)
    return u, J, x_

def small_strain(J):
    """Tensor de pequenas deformações ε = 1/2 (J + J^T)."""
    return 0.5*(J + J.T)

def mat_tensor_iso(E, nu):
    lam = E*nu/((1+nu)*(1-2*nu))
    mu  = E/(2*(1+nu))
    I = torch.eye(3, dtype=torch.get_default_dtype())
    C = torch.zeros(3,3,3,3, dtype=torch.get_default_dtype())
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    C[i,j,k,l] = lam*I[i,j]*I[k,l] + mu*(I[i,k]*I[j,l] + I[i,l]*I[j,k])
    return C, lam, mu

def stress_from_C_eps(C, eps):
    return torch.einsum('ijkl,kl->ij', C, eps)

def df_tensor_2nd(T, names=('x','y','z'), title=None):
    df = pd.DataFrame(T.detach().numpy()[:len(names),:len(names)],
                      index=[f'{n}' for n in names[:T.shape[0]]],
                      columns=[f'{n}' for n in names[:T.shape[1]]])
    if title: print(title)
    display(df)
    return df
    
def df_tensor_4th(C):
    idx = [(0,0),(1,1),(2,2),(1,2),(0,2),(0,1)]
    V = torch.zeros(6,6, dtype=C.dtype)
    for a,(i,j) in enumerate(idx):
        for b,(k,l) in enumerate(idx):
            V[a,b] = C[i,j,k,l]
    df = pd.DataFrame(V.detach().numpy(),
                      index=['xx','yy','zz','yz','xz','xy'],
                      columns=['xx','yy','zz','yz','xz','xy'])
    print('Tensor de material (Voigt, sem fator de 2 em cisalhamento):')
    display(df)
    return df

In [2]:
E = 210e9
nu = 0.30
eps0 = 1e-3

def u1(x):
    # Retorna vetor 1D [u_x]; usar stack preserva grad_fn
    return torch.stack([eps0*x[0]])

x1 = torch.tensor([2.0])
u, J, _ = grad_u(u1, x1)
eps = small_strain(J)[0:1,0:1]
sigma_xx = E*eps[0,0]
print('u(x) =', u)
print('du/dx =', float(J[0,0]))
print('ε_xx =', float(eps[0,0]))
print('σ_xx =', float(sigma_xx), 'Pa')

u(x) = tensor([0.0020], grad_fn=<StackBackward0>)
du/dx = 0.001
ε_xx = 0.001
σ_xx = 210000000.0 Pa


In [3]:
eps0 = 500e-6
def u2(p):
    x,y = p
    return torch.stack([eps0*x, -nu*eps0*y])

x2 = torch.tensor([1.0, 0.5])
u, J, _ = grad_u(u2, x2)
eps2 = small_strain(J)[0:2,0:2]
print('u(x,y)=', u)
df_tensor_2nd(eps2, names=('x','y'), title='ε (2D)')

Cps = (E/(1-nu**2))*torch.tensor([[1.,  nu, 0.],
                                   [nu, 1., 0.],
                                   [0.,  0., (1-nu)/2]])
eps_voigt = torch.stack([eps2[0,0], eps2[1,1], 2*eps2[0,1]])
sig_voigt = Cps @ eps_voigt
sigma2 = torch.tensor([[sig_voigt[0], sig_voigt[2]],
                       [sig_voigt[2], sig_voigt[1]]])
df_tensor_2nd(sigma2, names=('x','y'), title='σ (2D, tensão plana) [Pa]')


u(x,y)= tensor([ 5.0000e-04, -7.5000e-05], grad_fn=<StackBackward0>)
ε (2D)


,x,y
x,0.0005,0.00000
y,0.0000,-0.00015


σ (2D, tensão plana) [Pa]


,x,y
x,105000000.0,0.000000e+00
y,0.0,-2.200050e-09


,x,y
x,105000000.0,0.000000e+00
y,0.0,-2.200050e-09


In [4]:
alpha = 2e-4
omega = 1e-3

def u3(p):
    x,y,z = p
    return torch.stack([alpha*x - omega*y,
                        alpha*y + omega*x,
                        alpha*z])

x3 = torch.tensor([0.3, -0.2, 0.5])
u, J, _ = grad_u(u3, x3)
eps3 = small_strain(J)
W = 0.5*(J - J.T)
print('u(x,y,z)=', u)
df_tensor_2nd(J, title='Gradiente de deslocamento J')
df_tensor_2nd(eps3, title='ε (3D)')
df_tensor_2nd(W, title='Parte antissimétrica W (rotação)')

C, lam, mu = mat_tensor_iso(E, nu)
print(f'Constantes de Lamé: λ={lam:.3e} Pa, μ={mu:.3e} Pa')
df_tensor_4th(C)
sigma3 = stress_from_C_eps(C, eps3)
df_tensor_2nd(sigma3, title='σ (3D) [Pa]')

u(x,y,z)= tensor([0.0003, 0.0003, 0.0001], grad_fn=<StackBackward0>)
Gradiente de deslocamento J


,x,y,z
x,0.0002,-0.0010,0.0000
y,0.0010,0.0002,0.0000
z,0.0000,0.0000,0.0002


ε (3D)


,x,y,z
x,0.0002,0.0000,0.0000
y,0.0000,0.0002,0.0000
z,0.0000,0.0000,0.0002


Parte antissimétrica W (rotação)


...

Constantes de Lamé: λ=1.212e+11 Pa, μ=8.077e+10 Pa
Tensor de material (Voigt, sem fator de 2 em cisalhamento):


,xx,yy,zz,yz,xz,xy
xx,2.826923e+11,1.211538e+11,1.211538e+11,0.000000e+00,0.000000e+00,0.000000e+00
yy,1.211538e+11,2.826923e+11,1.211538e+11,0.000000e+00,0.000000e+00,0.000000e+00
zz,1.211538e+11,1.211538e+11,2.826923e+11,0.000000e+00,0.000000e+00,0.000000e+00
yz,0.000000e+00,0.000000e+00,0.000000e+00,8.076923e+10,0.000000e+00,0.000000e+00
xz,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.076923e+10,0.000000e+00
xy,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.076923e+10


σ (3D) [Pa]


,x,y,z
x,105000000.0,0.0,0.0
y,0.0,105000000.0,0.0
z,0.0,0.0,105000000.0


,x,y,z
x,105000000.0,0.0,0.0
y,0.0,105000000.0,0.0
z,0.0,0.0,105000000.0
